# DFIP: Decentralized Financial Insurance Protocol

In this tutorial, we build a probabilistic model for an insurance protocol
that must manage reserves against uncertain claim arrivals and sizes. We use
a **Poisson** distribution for claim frequency, a **LogNormal** distribution
for claim sizes, and compose them to estimate total loss distributions.

## Setup

In [ ]:
import numpy as np
from probflow import Poisson, LogNormal, Normal, ProbFlow

## Modeling Claim Arrivals

Insurance claims arrive randomly over time. The Poisson distribution is
the standard model for counting events in a fixed time period. We set
`lam=5` to model an average of 5 claims per day.

In [ ]:
claim_arrivals = Poisson(lam=5)
daily_counts = claim_arrivals.sample(30)
print(f"Claims per day (30 days): {daily_counts}")
print(f"Average daily claims: {np.mean(daily_counts):.2f}")

## Claim Arrival Probabilities

We compute the probability of observing specific numbers of claims.

In [ ]:
for k in range(0, 11):
    prob = claim_arrivals.pdf(k)
    print(f"  P(claims = {k:2d}) = {prob:.4f}")

## Probability of a Surge

What is the probability of receiving more than 8 claims in a single day?
We use the CDF: P(X > 8) = 1 - P(X ≤ 8).

In [ ]:
prob_surge = 1.0 - claim_arrivals.cdf(8)
print(f"P(claims > 8) = {prob_surge:.4f}")

## Modeling Claim Sizes

Individual claim sizes are typically right-skewed: most claims are small
but a few are very large. The LogNormal distribution captures this well.
Here `mu=7` and `sigma=1.5` model claims with a median around $1,097
(i.e., `exp(7)`).

In [ ]:
claim_size = LogNormal(mu=7, sigma=1.5)
sizes = claim_size.sample(1000)
print(f"Mean claim size: ${np.mean(sizes):,.2f}")
print(f"Median claim size: ${np.median(sizes):,.2f}")

## Extreme Claim Quantiles

The insurer needs to know how large claims can get. We compute
the 95th and 99th percentile claim sizes.

In [ ]:
p95 = claim_size.quantile(0.95)
p99 = claim_size.quantile(0.99)
print(f"95th percentile claim: ${p95:,.2f}")
print(f"99th percentile claim: ${p99:,.2f}")

## Total Daily Loss via Composition

The total daily loss depends on both the number of claims and their sizes.
We approximate this by composing the Poisson (frequency) and LogNormal
(severity) distributions using the product (`*`) operator. This gives
a distribution whose samples represent `count * size` scenarios.

In [ ]:
total_loss = claim_arrivals * claim_size
loss_samples = total_loss.sample(10000)
print(f"Mean daily total loss: ${np.mean(loss_samples):,.2f}")
print(f"Std daily total loss: ${np.std(loss_samples):,.2f}")

## Loss Quantiles for Reserve Planning

The protocol must hold sufficient reserves. We compute quantiles of
the total loss distribution to set reserve levels.

In [ ]:
for q in [0.50, 0.75, 0.90, 0.95, 0.99]:
    val = total_loss.quantile(q)
    print(f"  {int(q*100)}th percentile loss: ${val:,.2f}")

## Adding an Operating Cost Component

The protocol also has fixed operating costs with some noise. We add
a Normal distribution for costs to the total loss using `+`.

In [ ]:
operating_cost = Normal(loc=2000.0, scale=300.0)
total_expense = total_loss + operating_cost
expense_samples = total_expense.sample(10000)
print(f"Mean total daily expense: ${np.mean(expense_samples):,.2f}")

## DFIP Model Definition

We define the complete insurance protocol model using ProbFlow.

In [ ]:
with ProbFlow() as dfip_model:
    freq = Poisson(lam=5)
    dfip_model.add_distribution(freq, name="claim_frequency")

    severity = LogNormal(mu=7, sigma=1.5)
    dfip_model.add_distribution(severity, name="claim_severity")

    costs = Normal(loc=2000, scale=300)
    dfip_model.add_distribution(costs, name="operating_costs")

print(f"DFIP model components: {list(dfip_model.variables.keys())}")

## Querying the Model

We retrieve distributions from the model to run analyses.

In [ ]:
freq_dist = dfip_model.get_distribution("claim_frequency")
sev_dist = dfip_model.get_distribution("claim_severity")
print(f"Expected claims: {np.mean(freq_dist.sample(5000)):.2f}")
print(f"Median claim size: ${np.median(sev_dist.sample(5000)):,.2f}")

## Summary

In this tutorial we:
- Modeled claim arrivals with a **Poisson** distribution
- Modeled claim sizes with a **LogNormal** distribution
- Used `*` to compose frequency and severity into total loss
- Used `+` to add operating costs to total loss
- Computed **quantiles** for reserve planning
- Defined a complete model with the **ProbFlow** context manager